In [2]:
!pip install pyopencl

     |████████████████████████████████| 747kB 9.3MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=93cb2c3538a6580827cc5656e4bbf6940f3fd7bb76f117296f568ab5c577afd9
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pytools


In [3]:
import time as tm
import numpy as np
import pyopencl as cl
import math

In [8]:
K = 10000
localSize = 80
globalSize = math.ceil(K/localSize) * localSize
groups = math.ceil(K/localSize)


L = 2
np.random.seed(12)
host_vectors_x = np.random.uniform(low=-L, high=L, size=globalSize,).astype(np.float32)
np.random.seed(11)
host_vectors_y = np.random.uniform(low=-L, high=L, size=globalSize).astype(np.float32)
host_labels = np.zeros(globalSize).astype(np.float32)
host_group_sums = np.zeros(groups).astype(np.float32)

plataforms = cl.get_platforms()
print("Plataformas: ", plataforms)

gpu_devices = plataforms[0].get_devices(cl.device_type.GPU)
print("Devices: ", gpu_devices)

context = cl.Context(gpu_devices)
queue = cl.CommandQueue(context)

device_vectors_x = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf= host_vectors_x)
device_vectors_y = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf= host_vectors_y)


device_group_sums = cl.Buffer(context, cl.mem_flags.WRITE_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf= host_group_sums)
device_labels = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, host_labels.nbytes)

print(f"Iniciando algoritmo de Monte Carlo con K={K}")
print(f"Redondeando K a {globalSize}")
print(f"Se generarán {groups} grupos con {localSize} hilos cada uno\n")

program = cl.Program(context, 
"""
__kernel void mandelbrot(
    __global float *vectors_x,
     __global float *vectors_y,
     __global float *device_labels,
      __global float *group_sums)
{
  int global_id = get_global_id(0);
  int local_id = get_local_id(0);
  int group_id = get_group_id(0); 
  int local_size = get_local_size(0);

  float c_x = vectors_x[global_id];
  float c_y = vectors_y[global_id];

  float z_nx = 0;
  float z_ny = 0;
  int i = 0;
  float norm_zn = 0;
  float u, v;
  
  while (i < 10000)
	{
		// Primero obtenemos el z_(n+1)
    u = (z_nx * z_nx) - (z_ny * z_ny);
    v =  2 * z_nx * z_ny;
    z_nx = u + c_x;
    z_ny = v + c_y;

    norm_zn = (z_nx * z_nx) + (z_ny * z_ny);
    if (norm_zn >= 4)
	  {
		  i = 1000000;
      device_labels[global_id] = 1;
	  }
    i += 1;
	}

  // Ahora hacemos la reducción
  
  if(local_id == 0){
    for (int j = 0; j < local_size; j++) {
      group_sums[group_id] += device_labels[global_id + j];
    }
  }

}
"""
).build()


t0_GPU = tm.time()
program.mandelbrot(queue, (globalSize,), (localSize,) , device_vectors_x, device_vectors_y, device_labels, device_group_sums)
cl.enqueue_copy(queue, host_group_sums, device_group_sums)
mandelbrot_complement = float(np.sum(host_group_sums))
ratio = np.float32((globalSize - mandelbrot_complement)/ globalSize)
area = 4*L*L*ratio 
t1_GPU = tm.time()
print(f"\nAproximación superficie: {area}")
print(f"Error: {np.abs(area-1.50659177) }")
print(f"Tiempo de ejecución: {t1_GPU-t0_GPU}")

Plataformas:  [<pyopencl.Platform 'NVIDIA CUDA' at 0x3d9e760>]
Devices:  [<pyopencl.Device 'Tesla T4' on 'NVIDIA CUDA' at 0x3d9e800>]
Iniciando algoritmo de Monte Carlo con K=10000
Redondeando K a 10000
Se generarán 125 grupos con 80 hilos cada uno


Aproximación superficie: 1.5152000188827515
Error: 0.008608248882751468
Tiempo de ejecución: 0.0019130706787109375
